## load library

In [ ]:
#python
import random
from tqdm import tqdm, trange,notebook
import argparse
import json
import pickle
import re
import collections
import wandb


#pytorch
import torch
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)
import torch.nn.functional as F

#tokenizer
from konlpy.tag import Mecab
from nltk import FreqDist

#embedding
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi,BM25L,BM25Plus

#numpy
import numpy as np

#pandas
import pandas as pd

#dataset

from datasets import load_dataset,load_from_disk,load_metric,DatasetDict,Dataset,Features,Value,concatenate_datasets,Sequence

#transformer

from transformers import AutoTokenizer,AutoConfig, EvalPrediction, AutoModelForQuestionAnswering, TrainingArguments, Trainer, default_data_collator
from transformers import BertModel, BertPreTrainedModel,AdamW,get_linear_schedule_with_warmup,BertTokenizerFast,ElectraTokenizerFast

#utility

from trainer_qa import QuestionAnsweringTrainer
from utils_qa import postprocess_qa_predictions

## hyperparameter

In [ ]:
max_seq_length = 384 # 질문과 컨텍스트, special token을 합한 문자열의 최대 길이
max_length = 384
pad_to_max_length = True
doc_stride = 128 # 컨텍스트가 너무 길어서 나눴을 때 오버랩되는 시퀀스 길이
max_train_samples = 16
max_val_samples = 16
preprocessing_num_workers = 4
batch_size = 8
num_train_epochs = 1
n_best_size = 20
max_answer_length = 30
data_collator = default_data_collator
dense_batch_size = 16
k = 20
save_strategy='no'

## fixed seed

In [ ]:
#torch seed
torch.manual_seed(30)
torch.cuda.manual_seed(30)

#numpy seed
np.random.seed(30)

#python seed
random.seed(30)

## read dataset

In [ ]:
datasets_base = load_from_disk("input/data/data/train_dataset")
datasets_korquad = load_dataset("squad_kor_v1")

Reusing dataset squad_kor_v1 (/opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7)


In [ ]:
datasets_base

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 240
    })
})

In [ ]:
datasets_korquad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})

In [ ]:
# train & valid set
train_datasets_base = datasets_base['train']
train_datasets_korquad = datasets_korquad['train']
valid_datasets_base = datasets_base['validation']
valid_datasets_korquad = datasets_korquad['validation']

## prepare ai hub dataset

In [ ]:
#read ai hub data

with open('ko_nia_normal_squad_all.json' , 'r', encoding='utf-8') as f:
    dataset_koquad = json.load(f)

In [ ]:
dataset_koquad = dataset_koquad['data']

In [ ]:
len(dataset_koquad)

47314

In [ ]:
#first preprocess ai hub dataset

data_list = []

data_dict = {}


for data in dataset_koquad:
    
    context = data['paragraphs'][0]['context']
    
    for qna in data['paragraphs'][0]['qas']:
        
        answers = {}
        
        question = qna['question']
        answer = qna['answers'][0]
        
        answers['answer_start'] = [answer['answer_start']]
        answers['text'] = [answer['text']]
        
        data_dict['context'] = context
        data_dict['question'] = question
        data_dict['answers'] = answers
        
        data_list.append(data_dict)
        
        data_dict = {}

In [ ]:
len(data_list)

243425

In [ ]:
data_list[0]

{'context': "한국청소년단체협의회와 여성가족부는 22일부터 28일까지 서울과 충북 괴산에서 '국제청소년포럼'을 연다고 21일 밝혔다. 한국 미국 캐나다 호주 등 전 세계 32개국 75여명의 대학생, 청소년들이 모여 전 세계적 현안문제에 대한 대안과 해결책을 모색하는 자리다. 이번 포럼의 주제는 '청소년과 뉴미디어'다. 스마트폰 SNS 태블릿PC 등 새로운 커뮤니케이션 매체인 '뉴미디어'에 대한 성찰과 문제점에 대해 토론한다. 기조강연을 시작으로 국가별 주제관련 사례발표, 그룹 토론 및 전체총회, '청소년선언문' 작성 및 채택 등 다양한 프로그램을 운영한다. 개회식은 22일 서울 방화동에 있는 국제청소년센터 국제회의장에서 한다. 전 세계 32개국 대학생ㆍ청소년 참가자와 전국의 청소년기관단체장과 청소년지도자 여성가족부 주한외교사절 등 100여명이 참석할 예정이다. 23일에는 유엔미래포럼 박영숙 대표가 '뉴미디어의 균형 있는 발전을 위한 청소년의 역할'에 대해 기조강연을 한다. 뉴미디어의 올바른 활용방안과 청소년문화의 형성에 대해 설명할 계획이다. 27일 폐회식에서는 '청소년선언문'을 채택한다. 선언문에는 전 세계적으로 뉴미디어의 바람직한 발전을 촉구하며 각국 청년들이 함께 실천할 수 있는 내용 등이 담길 예정이다. 한국청소년단체협의회는 포럼이 끝난 뒤 UN 등 국제기구와 참가자 각국 정부 등 국제사회에 선언문을 전달할 예정이다.",
 'question': "서울과 충북 괴산에서 '국제청소년포럼'을 여는 곳은?",
 'answers': {'answer_start': [0], 'text': ['한국청소년단체협의회와 여성가족부']}}

In [ ]:
#second preprocess ai hub dataset

context_list = []
question_list = []
answer_list = []

for data in data_list:
    
    context,question,answers = data.values()
    
    context_list.append(context)
    question_list.append(question)
    answer_list.append(answers)

In [ ]:
len(context_list)

243425

In [ ]:
len(question_list)

243425

In [ ]:
len(answer_list)

243425

## concatenation ai hub dataset

In [ ]:
train_datasets_korquad = concatenate_datasets([train_datasets_korquad,valid_datasets_korquad])

In [ ]:
korquad_context_list = train_datasets_korquad['context']
korquad_question_list = train_datasets_korquad['question']
korquad_answer_list = train_datasets_korquad['answers']

In [ ]:
len(korquad_context_list)

66181

In [ ]:
len(korquad_question_list)

66181

In [ ]:
len(korquad_answer_list)

66181

In [ ]:
base_context_list = train_datasets_base['context']
valid_context_list = valid_datasets_base['context']
base_question_list =train_datasets_base['question']
base_answer_list = train_datasets_base['answers']

In [ ]:
len(base_context_list)

3952

In [ ]:
len(base_question_list)

3952

In [ ]:
len(base_answer_list)

3952

In [ ]:
context_list.extend(korquad_context_list)
question_list.extend(korquad_question_list)
answer_list.extend(korquad_answer_list)

In [ ]:
context_list.extend(base_context_list)
question_list.extend(base_question_list)
answer_list.extend(base_answer_list)

In [ ]:
context_list.extend(valid_context_list)

In [ ]:
len(context_list)

313798

In [ ]:
len(question_list)

313558

In [ ]:
len(answer_list)

313558

## create top20 bm25 score dataset

In [ ]:
corpus = context_list

In [ ]:
clean_corpus = get_clean_wikipedia_text_v4(corpus)

In [ ]:
clean_corpus = list(set(clean_corpus))

In [ ]:
len(clean_corpus)

59528

In [ ]:
train_datasets_base

Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
    num_rows: 3952
})

In [ ]:
ground_truth_context_list = train_datasets_base['context']

In [ ]:
ground_truth_context_list = get_clean_wikipedia_text_v4(ground_truth_context_list)

In [ ]:
gt_id_list = []

for index,gt in enumerate(ground_truth_context_list):
    
    for idx,document in enumerate(clean_corpus):
        if gt == document:
            gt_id_list.append(idx)
            break
        

In [ ]:
question_list = train_datasets_base['question']

In [ ]:
len(question_list)

3952

In [ ]:
query_bm25_list = get_relevant_doc_bm25(clean_corpus,question_list,20)

In [ ]:
with open('query_bm25_list_train.pkl', 'wb') as f:
    pickle.dump(query_bm25_list, f)

In [ ]:
with open('query_bm25_list_train.pkl', 'rb') as f:
    query_bm25_list = pickle.load(f)

In [ ]:
context_list = []

for idx,a in notebook.tqdm(enumerate(query_bm25_list)):
    
    _,context_id,_ = a

    context = []
    
    context_id = context_id.numpy()
    
    if gt_id_list[idx] in list(context_id):
        
        np.random.shuffle(context_id)
        
        for idx in context_id:
            context.append(clean_corpus[idx])
            
        context_string = ' '.join(context)

        context_list.append(context_string)
    
    else:
        
        context_id = list(context_id[:19])
        context_id.append(gt_id_list[idx])
        
        context_id = np.array(context_id)
        
        np.random.shuffle(context_id)
        
        for idx in context_id:
            context.append(clean_corpus[idx])
            
        context_string = ' '.join(context)

        context_list.append(context_string)

In [ ]:
base_answer_list = train_datasets_base['answers']

In [ ]:
new_answer_list = []

for answer,context in zip(base_answer_list,context_list):
    
    new_answer = {}
    
    answer_text = answer['text'][0]
    
    answer_start = [context.index(answer_text)]
    
    new_answer['answer_start']=answer_start
    new_answer['text'] = [answer_text]
    
    new_answer_list.append(new_answer)

In [ ]:
base_question_list =train_datasets_base['question']

In [ ]:
#finally exchange dataset class
data_dict = {'answers':new_answer_list,'question':base_question_list,'context':context_list}
    
f = Features({'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)})
    
train_datasets_final = DatasetDict({'train': Dataset.from_dict(data_dict, features=f)})

In [ ]:
train_datasets_base = train_datasets_final['train']

In [ ]:
train_datasets_base #train set

Dataset({
    features: ['context', 'question', 'answers'],
    num_rows: 3952
})

## top20 bm25 score valid set

In [ ]:
valid_context_list = valid_datasets_base['context']
valid_question_list =valid_datasets_base['question']
valid_answer_list = valid_datasets_base['answers']
valid_id_list = valid_datasets_base['id']

In [ ]:
ground_truth_valid_context_list = get_clean_wikipedia_text_v4(valid_context_list)

In [ ]:
gt_id_valid_list = []

for index,gt in enumerate(ground_truth_valid_context_list):
    
    for idx,document in enumerate(clean_corpus):
        if gt == document:
            gt_id_valid_list.append(idx)
            break

In [ ]:
query_bm25_valid_list = get_relevant_doc_bm25(clean_corpus,valid_question_list,20)

In [ ]:
with open('query_bm25_list_valid.pkl', 'wb') as f:
    pickle.dump(query_bm25_valid_list, f)

In [ ]:
with open('query_bm25_list_valid.pkl', 'rb') as f:
    query_bm25_valid_list = pickle.load(f)

In [ ]:
new_context_list = []

for idx,a in notebook.tqdm(enumerate(query_bm25_valid_list)):
    
    _,context_id,_ = a

    context = []
    
    context_id = context_id.numpy()
    
    if gt_id_valid_list[idx] in list(context_id):
        
        np.random.shuffle(context_id)
        
        for idx in context_id:
            context.append(clean_corpus[idx])
            
        context_string = ' '.join(context)

        new_context_list.append(context_string)
    
    else:
        
        context_id = list(context_id[:19])
        context_id.append(gt_id_valid_list[idx])
        
        context_id = np.array(context_id)
        
        np.random.shuffle(context_id)
        
        for idx in context_id:
            context.append(clean_corpus[idx])
            
        context_string = ' '.join(context)

        new_context_list.append(context_string)

In [ ]:
new_answer_list = []

for answer,context in zip(valid_answer_list,new_context_list):
    
    new_answer = {}
    
    answer_text = answer['text'][0]
    
    answer_start = [context.index(answer_text)]
    
    new_answer['answer_start']=answer_start
    new_answer['text'] = [answer_text]
    
    new_answer_list.append(new_answer)

In [ ]:
data_dict = {'answers':new_answer_list,'question':valid_question_list,'context':new_context_list,'id':valid_id_list}
    
f = Features({'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None),
'id': Value(dtype='string', id=None)
             })
    
valid_datasets_final = DatasetDict({'valid': Dataset.from_dict(data_dict, features=f)})

In [ ]:
valid_datasets_base= valid_datasets_final['valid']

In [ ]:
valid_datasets_base #validation set

Dataset({
    features: ['context', 'question', 'answers', 'id'],
    num_rows: 240
})

## check dataset

In [ ]:
print(train_datasets_base[0]) #train set

{'answers': {'answer_start': [0], 'text': ['한국청소년단체협의회와 여성가족부']}, 'question': "서울과 충북 괴산에서 '국제청소년포럼'을 여는 곳은?", 'context': "한국청소년단체협의회와 여성가족부는 22일부터 28일까지 서울과 충북 괴산에서 '국제청소년포럼'을 연다고 21일 밝혔다. 한국 미국 캐나다 호주 등 전 세계 32개국 75여명의 대학생, 청소년들이 모여 전 세계적 현안문제에 대한 대안과 해결책을 모색하는 자리다. 이번 포럼의 주제는 '청소년과 뉴미디어'다. 스마트폰 SNS 태블릿PC 등 새로운 커뮤니케이션 매체인 '뉴미디어'에 대한 성찰과 문제점에 대해 토론한다. 기조강연을 시작으로 국가별 주제관련 사례발표, 그룹 토론 및 전체총회, '청소년선언문' 작성 및 채택 등 다양한 프로그램을 운영한다. 개회식은 22일 서울 방화동에 있는 국제청소년센터 국제회의장에서 한다. 전 세계 32개국 대학생ㆍ청소년 참가자와 전국의 청소년기관단체장과 청소년지도자 여성가족부 주한외교사절 등 100여명이 참석할 예정이다. 23일에는 유엔미래포럼 박영숙 대표가 '뉴미디어의 균형 있는 발전을 위한 청소년의 역할'에 대해 기조강연을 한다. 뉴미디어의 올바른 활용방안과 청소년문화의 형성에 대해 설명할 계획이다. 27일 폐회식에서는 '청소년선언문'을 채택한다. 선언문에는 전 세계적으로 뉴미디어의 바람직한 발전을 촉구하며 각국 청년들이 함께 실천할 수 있는 내용 등이 담길 예정이다. 한국청소년단체협의회는 포럼이 끝난 뒤 UN 등 국제기구와 참가자 각국 정부 등 국제사회에 선언문을 전달할 예정이다."}


In [ ]:
print(train_datasets_korquad[0]) #korquad train set

{'answers': {'answer_start': [54], 'text': ['교향곡']}, 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.', 'id': '6566495-0-0', 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?', 'title': '파우스트_서곡'}


In [ ]:
print(valid_datasets_base[111]) #valid set

{'title': '교황 그레고리오 4세', 'context': '한편 그레고리오 4세는 로마의 건축 발전에 크게 기여하였다. 833년 그레고리오 4세는 산 마르코 성당을 비잔티움 양식의 모자이크로 벽을 장식하는 등 완전히 새로운 형태로 탈바꿈하는 등 로마 시내의 많은 성당을 보수하거나 신축하였다. 그는 성 베드로 대성전의 안마당을 새로 포장했으며, 교황 그레고리오 1세의 유해를 대성전 안에 새로 설치한 부속 경당 안에 이장하였다. 그리고 로마의 카타콤베에서 성 세바스티아노와 성 티부르시오, 성 고르고니오의 유해를 다른 곳으로 이장하였다. 더불어 산타 마리아 인 트라스테베레 성당의 제대를 높이 만들고, 성당과 가까운 곳에 수도원을 세우도록 지시하였다. \\n\\n그레고리오 4세는 또한 교황 레오 3세 재위기간 중에 훼손되었던 트라야나 수도를 보수하였다. 841년에는 사라센족의 침입에 대비하기 위해 오스티아 항구를 다시 지어 요새화하였다. 동시에 그는 포폴리아 가도 언저리에 있는 갈레리아 개척지를 복구하였으며, 테베레 강가를 따라 왼쪽으로 드라라고 불리는 새로운 개척지를 구축하였다. 이곳은 로마로부터 오스티엔시스 가도를 따라 약 17킬로미터 정도 떨어진 곳이었다. 이는 교회 역사상 처음으로 교황이 자신의 관리구역을 토지 개발한 사례가 되었다. \\n\\n그레고리오 4세의 재위기간 동안 동로마 제국에서는 성화상 논쟁이 종지부를 찍었으며, 교황 자신은 프랑크 제국 영역인 라인 강 인근에서 모든 성인 대축일 기념 행사를 성대하게 개최하였다. 그레고리오 4세는 또한 832년에 안스가리오를 함부르크와 브레멘의 주교로 서임했으며, 유럽의 북동부 지역을 담당하는 교황 사절을 겸임하도록 하였다. \\n\\n844년 1월 25일 그레고리오 4세는 선종하였으며, 시신은 성 베드로 대성전에 안장되었다.', 'question': '교황에 의해 새롭게 토지 개발된 최초의 관리지역은?', 'id': 'mrc-1-000297', 'answers': {'answer_start': [493]

In [ ]:
print(valid_datasets_korquad[5]) #korquad valid set

{'answers': {'answer_start': [87], 'text': ['임종석']}, 'context': '1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률위반)으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일~20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다.', 'id': '6332405-0-0', 'question': '1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?', 'title': '임종석'}


In [ ]:
train_datasets_base.features #train feature

{'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}

In [ ]:
train_datasets_korquad.features #korquad train feature

{'id': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}

In [ ]:
valid_datasets_base.features #valid feature

{'__index_level_0__': Value(dtype='int64', id=None),
 'answers': {'answer_start': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
  'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)},
 'context': Value(dtype='string', id=None),
 'document_id': Value(dtype='int64', id=None),
 'id': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None)}

In [ ]:
valid_datasets_korquad.features #valid feature

{'id': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}

## load metric

In [ ]:
metric = load_metric('squad')

## define preprocess function

In [ ]:
def prepare_train_features(examples):
    # 주어진 텍스트를 토크나이징 한다. 이 때 텍스트의 길이가 max_seq_length를 넘으면 stride만큼 슬라이딩하며 여러 개로 쪼갬.
    # 즉, 하나의 example에서 일부분이 겹치는 여러 sequence(feature)가 생길 수 있음.
    tokenized_examples = mrc_tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",  # max_seq_length까지 truncate한다. pair의 두번째 파트(context)만 잘라냄.
        max_length=max_seq_length,
        stride=doc_stride,
        return_overflowing_tokens=True, # 길이를 넘어가는 토큰들을 반환할 것인지
        return_offsets_mapping=True,  # 각 토큰에 대해 (char_start, char_end) 정보를 반환한 것인지
        padding="max_length",
    )
    
    # example 하나가 여러 sequence에 대응하는 경우를 위해 매핑이 필요함.
    overflow_to_sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # offset_mappings으로 토큰이 원본 context 내 몇번째 글자부터 몇번째 글자까지 해당하는지 알 수 있음.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # 정답지를 만들기 위한 리스트
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(mrc_tokenizer.cls_token_id)
        
        # 해당 example에 해당하는 sequence를 찾음.
        sequence_ids = tokenized_examples.sequence_ids(i)
        
        # sequence가 속하는 example을 찾는다.
        example_index = overflow_to_sample_mapping[i]
        answers = examples["answers"][example_index]
        
        # 텍스트에서 answer의 시작점, 끝점
        answer_start_offset = answers["answer_start"][0]
        answer_end_offset = answer_start_offset + len(answers["text"][0])

        # 텍스트에서 현재 span의 시작 토큰 인덱스
        token_start_index = 0
        while sequence_ids[token_start_index] != 1:
            token_start_index += 1
        
        # 텍스트에서 현재 span 끝 토큰 인덱스
        token_end_index = len(input_ids) - 1
        while sequence_ids[token_end_index] != 1:
            token_end_index -= 1

        # answer가 현재 span을 벗어났는지 체크
        if not (offsets[token_start_index][0] <= answer_start_offset and offsets[token_end_index][1] >= answer_end_offset):
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # token_start_index와 token_end_index를 answer의 시작점과 끝점으로 옮김
            while token_start_index < len(offsets) and offsets[token_start_index][0] <= answer_start_offset:
                token_start_index += 1
            tokenized_examples["start_positions"].append(token_start_index - 1)
            while offsets[token_end_index][1] >= answer_end_offset:
                token_end_index -= 1
            tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
def prepare_validation_features(examples):
    tokenized_examples = mrc_tokenizer(
        examples['question'],
        examples['context'],
        truncation="only_second",
        max_length=max_seq_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1

        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

## cleansing competition data

In [ ]:
train_datasets_base

Dataset({
    features: ['context', 'question', 'answers'],
    num_rows: 3952
})

In [ ]:
valid_datasets_base

Dataset({
    features: ['context', 'question', 'answers', 'id'],
    num_rows: 240
})

In [ ]:
base_context_list = train_datasets_base['context']
base_question_list =train_datasets_base['question']
base_answer_list = train_datasets_base['answers']

In [ ]:
clean_base_context_list = get_clean_wikipedia_text_v4(base_context_list)

In [ ]:
new_answer_list = []

for answer,context in zip(base_answer_list,clean_base_context_list):
    
    new_answer = {}
    
    answer_text = answer['text'][0]
    
    answer_start = [context.index(answer_text)]
    
    new_answer['answer_start']=answer_start
    new_answer['text'] = [answer_text]
    
    new_answer_list.append(new_answer)

In [ ]:
data_dict = {'answers':new_answer_list,'question':base_question_list,'context':clean_base_context_list}
    
f = Features({'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)})
    
train_datasets_final = DatasetDict({'train': Dataset.from_dict(data_dict, features=f)})

In [ ]:
train_datasets_base = train_datasets_final['train']

In [ ]:
train_datasets_base

Dataset({
    features: ['context', 'question', 'answers'],
    num_rows: 3952
})

## cleansing competition valid data

In [ ]:
base_context_list = valid_datasets_base['context']
base_question_list =valid_datasets_base['question']
base_answer_list = valid_datasets_base['answers']
base_id_list = valid_datasets_base['id']

In [ ]:
clean_base_context_list = get_clean_wikipedia_text_v4(base_context_list)

In [ ]:
new_answer_list = []

for answer,context in zip(base_answer_list,clean_base_context_list):
    
    new_answer = {}
    
    answer_text = answer['text'][0]
    
    answer_start = [context.index(answer_text)]
    
    new_answer['answer_start']=answer_start
    new_answer['text'] = [answer_text]
    
    new_answer_list.append(new_answer)

In [ ]:
data_dict = {'answers':new_answer_list,'question':base_question_list,'context':clean_base_context_list,'id':base_id_list}
    
f = Features({'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None),
'id': Value(dtype='string', id=None)
             })
    
valid_datasets_final = DatasetDict({'valid': Dataset.from_dict(data_dict, features=f)})

In [ ]:
valid_datasets_base = valid_datasets_final['valid']

In [ ]:
valid_datasets_base

Dataset({
    features: ['context', 'question', 'answers', 'id'],
    num_rows: 240
})

## tokenizing & preprocessing mrc dataset

In [ ]:
#tokenizer

mrc_model_name = 'monologg/koelectra-base-v3-discriminator'
mrc_tokenizer = AutoTokenizer.from_pretrained(mrc_model_name)

#mecab = Mecab()

#def tokenize(text):
    #return mecab.morphs(text)

#mrc_tokenizer = tokenize
#dense_tokenizer = tokenize

In [ ]:
#preprocessing train dataset competition

column_names = train_datasets_base.column_names

train_dataset = train_datasets_base.map(
            prepare_train_features,
            batched=True,
            num_proc=preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=True,
)

In [ ]:
#preprocessing validation dataset competition

column_names = valid_datasets_base.column_names

eval_dataset = valid_datasets_base.map(
            prepare_validation_features,
            batched=True,
           num_proc=preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=True,
      )

In [ ]:
len(train_dataset)

80311

In [ ]:
len(eval_dataset)

533

In [ ]:
train_dataset

Dataset({
    features: ['attention_mask', 'end_positions', 'input_ids', 'start_positions', 'token_type_ids'],
    num_rows: 186544
})

In [ ]:
eval_dataset

Dataset({
    features: ['attention_mask', 'example_id', 'input_ids', 'offset_mapping', 'token_type_ids'],
    num_rows: 11487
})

## concatenation dataset

In [ ]:
#concatenation for retraining

train_datasets_base = concatenate_datasets([train_datasets_base,valid_datasets_base])

train_datasets_korquad = concatenate_datasets([train_datasets_korquad,valid_datasets_korquad])

In [ ]:
train_datasets_base

Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 4192
})

In [ ]:
train_datasets_korquad

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 66181
})

In [ ]:
#preprocesing train dataset

column_names = train_datasets_base.column_names

train_dataset = train_datasets_base.map(
            prepare_train_features,
            batched=True,
            num_proc=preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=True,
)

Loading cached processed dataset at input/data/data/train_dataset/train/cache-e4a4143b9734ad1d.arrow


Loading cached processed dataset at input/data/data/train_dataset/train/cache-a6046d86ae6d2246.arrow


Loading cached processed dataset at input/data/data/train_dataset/train/cache-d39d153864f9a2a5.arrow


Loading cached processed dataset at input/data/data/train_dataset/train/cache-a0c48ef3437204c0.arrow


In [ ]:
#preprocessing korquad dataset

column_names = train_datasets_korquad.column_names

train_dataset_korquad = train_datasets_korquad.map(
            prepare_train_features,
            batched=True,
            num_proc=preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=True,
)

Loading cached processed dataset at /opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7/cache-6c3a160c20cac49b.arrow


Loading cached processed dataset at /opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7/cache-5e9601b1774289ed.arrow


Loading cached processed dataset at /opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7/cache-2ebd36ebf6ce3745.arrow


Loading cached processed dataset at /opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7/cache-7d4ea920bd238c37.arrow


In [ ]:
len(train_dataset)

8572

In [ ]:
train_dataset

Dataset({
    features: ['attention_mask', 'end_positions', 'input_ids', 'start_positions', 'token_type_ids'],
    num_rows: 7836
})

In [ ]:
len(train_dataset_korquad)

76737

In [ ]:
train_dataset_korquad

Dataset({
    features: ['attention_mask', 'end_positions', 'input_ids', 'start_positions', 'token_type_ids'],
    num_rows: 76737
})

In [ ]:
#full dataset

train_dataset = concatenate_datasets([train_dataset,train_dataset_korquad])
len(train_dataset)

85309

In [ ]:
train_dataset

Dataset({
    features: ['attention_mask', 'end_positions', 'input_ids', 'start_positions', 'token_type_ids'],
    num_rows: 85309
})

## Dense embedding retrieval

## prepare dataset

In [ ]:
# tokenizer

dense_encoder_name = 'kykim/bert-kor-base'
dense_tokenizer = BertTokenizerFast.from_pretrained(dense_encoder_name)

In [ ]:
#dataset

q_seqs = dense_tokenizer(train_datasets_base['question'],padding='max_length',truncation=True, return_tensors='pt')

p_seqs = dense_tokenizer(train_datasets_base['context'],padding='max_length',truncation=True, return_tensors='pt')

In [ ]:
train_dataset_dense = TensorDataset(p_seqs['input_ids'],p_seqs['attention_mask'],p_seqs['token_type_ids'],
                             q_seqs['input_ids'],q_seqs['attention_mask'],q_seqs['token_type_ids'])

## define dense encoder class

In [ ]:
#define dense retrieval class

class DenseRetrieval(BertPreTrainedModel):
    def __init__(self,config):
        super(DenseRetrieval,self).__init__(config)
        
        self.bert = BertModel(config)
        self.init_weights()
        
    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        
        outputs = self.bert(input_ids,attention_mask=attention_mask, token_type_ids=token_type_ids)
        
        pooled_output = outputs[1]
        
        return pooled_output

In [ ]:
p_encoder = DenseRetrieval.from_pretrained(dense_encoder_name).cuda()
q_encoder = DenseRetrieval.from_pretrained(dense_encoder_name).cuda()

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing DenseRetrieval: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing DenseRetrieval from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DenseRetrieval from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing DenseRetrieval

In [ ]:
#load saved model

p_encoder = torch.load('p_encoder_kobert_full_ai_hub.pth').cuda()
q_encoder = torch.load('q_encoder_kobert_full_ai_hub.pth').cuda()

In [ ]:
if torch.cuda.is_available():
    print('GPU enabled')

GPU enabled


## training Dense encoder

In [ ]:
def train(args, dataset, p_model, q_model):
    
    #dataloader
    
    train_sampler = RandomSampler(dataset)
    train_dataloader = DataLoader(dataset, sampler = train_sampler, batch_size = args.per_device_train_batch_size)
    
    #optimizer
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params':[p for n,p in p_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay':args.weight_decay},
        {'params':[p for n,p in p_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay':0.0},
        {'params':[p for n,p in q_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay':args.weight_decay},
        {'params':[p for n,p in q_model.named_parameters() if any(nd in n for nd in no_decay)],'weight_decay':0.0}
    ]
    
    optimizer = AdamW(optimizer_grouped_parameters, lr = args.learning_rate, eps = args.adam_epsilon)
    t_total = len(train_dataloader) // args.gradient_accumulation_steps*args.num_train_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=args.warmup_steps,num_training_steps=t_total)
    
    global_step = 0
    
    p_model.zero_grad()
    q_model.zero_grad()
    torch.cuda.empty_cache()
    
    train_iterator = trange(int(args.num_train_epochs), desc='Epoch')
    
    for _ in train_iterator:
        
        epoch_iterator = notebook.tqdm(train_dataloader,desc='Iteration')
        
        for step,batch in enumerate(epoch_iterator):
            #q_encoder.train()
            #p_encoder.train()
            q_model.train()
            p_model.train()
            
            if torch.cuda.is_available():
                batch = tuple(t.cuda() for t in batch)
                
            p_inputs = {'input_ids':batch[0],
                       'attention_mask':batch[1],
                       'token_type_ids':batch[2]}
            
            q_inputs = {'input_ids':batch[3],
                       'attention_mask':batch[4],
                       'token_type_ids':batch[5]}
            
            p_outputs = p_model(**p_inputs) #(batch_size,emb_dim) #현재 question이랑 대응하는 passage 1개(positive sample)+나머지는 대응하지 않는 batchsize-1개(negative sample)
            q_outputs = q_model(**q_inputs) #(batch_size,emb_dim)
            
            #in-batch negative?? negative sample은 minimize하고 positive sample은 maximize하는 방법?
            #이건 생각 좀 많이 해봐야겠는데.. 
            sim_scores = torch.matmul(q_outputs, torch.transpose(p_outputs,0,1))
            
            #diagonal한 위치에 존재하는 positive sample
            targets = torch.arange(0,args.per_device_train_batch_size).long()
            
            if torch.cuda.is_available():
                targets = targets.to('cuda')
                
            sim_scores = F.log_softmax(sim_scores,dim=1)
            
            loss = F.nll_loss(sim_scores,targets) 
            
            print(loss)
            
            loss.backward()
            optimizer.step()
            scheduler.step()
            q_model.zero_grad()
            p_model.zero_grad()
            global_step += 1
            
            torch.cuda.empty_cache()
            
    return p_model,q_model

In [ ]:
args = TrainingArguments(output_dir='dense_retireval',
                        evaluation_strategy='epoch',
                        learning_rate=2e-5,
                        per_device_train_batch_size=16,
                        per_device_eval_batch_size=16,
                        num_train_epochs=2,
                        weight_decay=0.01,
                        save_strategy='no')

In [ ]:
p_encoder,q_encoder = train(args, train_dataset_dense,p_encoder,q_encoder)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

tensor(27.7413, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(14.1644, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(13.1647, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(16.9471, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.1358, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.8762, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(7.9383, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.6852, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.1457, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.1874, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(8.6159, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(7.1146, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.9234, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.9673, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.5149, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.2167, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.2124, device='cuda:0', grad_fn=<NllLossBac

ValueError: Expected input batch_size (6) to match target batch_size (16).

In [ ]:
#save dense encdoer

torch.save(p_encoder,'p_encoder_kobert_full_ai_hub.pth')
torch.save(q_encoder,'q_encoder_kobert_full_ai_hub.pth')

## Evaluation dense encoder

In [ ]:
valid_corpus = list(example['context'] for example in valid_datasets_base)

In [ ]:
eval_query = valid_datasets_base['question']
eval_ground_truth = valid_datasets_base['context']

In [ ]:
eval_q_seqs = dense_tokenizer(eval_query,padding='max_length',truncation=True, return_tensors='pt').to('cuda')
eval_p_seqs = dense_tokenizer(eval_ground_truth,padding='max_length',truncation=True, return_tensors='pt').to('cuda')

In [ ]:
with torch.no_grad():
    p_encoder.eval()
    q_encoder.eval()
    
    q_embs = q_encoder(**eval_q_seqs).to('cpu')
    
    p_embs = p_encoder(**eval_p_seqs).to('cpu')

    print(p_embs.size(),q_embs.size()) #(passage 개수, emb_size),(question개수,emb_size)

torch.Size([240, 768]) torch.Size([240, 768])


In [ ]:
#find top-10

accuracy = 0.0
k = 10

for i,q_emb in enumerate(q_embs):
    
    dot_product_scores = torch.matmul(q_emb,torch.transpose(p_embs,0,1)) 
    rank= torch.sort(dot_product_scores,descending=True).indices
    
    for j in range(k):
        if valid_corpus[rank[j]] == eval_ground_truth[i]:
            accuracy += 1
            break

eval_accuracy = accuracy/len(eval_query)

In [ ]:
print(eval_accuracy)

0.9791666666666666


## Define MRC model

In [ ]:
#load model

mrc_model = torch.load('mrc_model_koelectra_base_v3_discrim_aihub_korquad_save.pth')

FileNotFoundError: [Errno 2] No such file or directory: 'mrc_model_koelectra_base_v3_discrim_aihub_korquad_save.pth'

In [ ]:
def post_processing_function(examples, features, predictions,training_args):
    # Post-processing: we match the start logits and end logits to answers in the original context.
    predictions = postprocess_qa_predictions(
        examples=examples,
        features=features,
        predictions=predictions,
        version_2_with_negative=False,
        n_best_size=n_best_size,
        max_answer_length=max_answer_length,
        null_score_diff_threshold=0.0,
        output_dir=training_args.output_dir,
        is_world_process_zero=trainer.is_world_process_zero(),
    )
    
    # Format the result to the format the metric expects.
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in predictions.items()]
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in valid_datasets_base] #change dataset name
    return EvalPrediction(predictions=formatted_predictions, label_ids=references)

In [ ]:
def compute_metrics(p: EvalPrediction):
    return metric.compute(predictions=p.predictions, references=p.label_ids)

In [ ]:
mrc_model_name = 'monologg/koelectra-base-v3-discriminator'
mrc_tokenizer = AutoTokenizer.from_pretrained(mrc_model_name)

In [ ]:
config = AutoConfig.from_pretrained(
    mrc_model_name
)

mrc_model = AutoModelForQuestionAnswering.from_pretrained(
    mrc_model_name,
    config=config
)

file koelectra-base-v3-finetuned-korquad/config.json not found


OSError: Can't load config for 'koelectra-base-v3-finetuned-korquad'. Make sure that:

- 'koelectra-base-v3-finetuned-korquad' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'koelectra-base-v3-finetuned-korquad' is the correct path to a directory containing a config.json file



In [ ]:
#train argument

training_args = TrainingArguments(
    output_dir="outputs",
    do_train=True, 
    do_eval=True, 
    learning_rate=3e-5,
    fp16=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy='no',
    run_name='yun8'
    #eval_steps=100,
    #evaluation_strategy="steps"
)

In [ ]:
#training MRC model with competition data
wandb.login()
wandb.init()

trainer =  QuestionAnsweringTrainer(
    model=mrc_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    eval_examples=valid_datasets_base,
    data_collator=data_collator,
    tokenizer=mrc_tokenizer,
    post_process_function=post_processing_function,
    compute_metrics=compute_metrics,
)

wandb.finish()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


## train and evaluation MRC model

In [ ]:
train_result = trainer.train() #training|

Step,Training Loss
500,1.364600
1000,1.408700
1500,1.416600
2000,1.378900
2500,1.377800
3000,1.388800
3500,1.385000
4000,1.391600
4500,1.395700
5000,1.378800


In [ ]:
train_result

TrainOutput(global_step=69954, training_loss=1.3339420855957018, metrics={'train_runtime': 9320.4686, 'train_samples_per_second': 7.505, 'total_flos': 1.4484037044621312e+17, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 779022, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 6965806, 'init_mem_gpu_peaked_delta': 512, 'train_mem_cpu_alloc_delta': 425329921, 'train_mem_gpu_alloc_delta': 899784192, 'train_mem_cpu_peaked_delta': 7275759, 'train_mem_gpu_peaked_delta': 3590787072})

In [ ]:
metrics = trainer.evaluate() #evaluation|

wandb: WARNING Step must only increase in log calls.  Step 69954 < 69955; dropping {'train/exact_match': 37.083333333333336, 'train/f1': 44.3331450956451, 'train/epoch': 3.0}.


In [ ]:
#save model

torch.save(mrc_model,'mrc_model_koelectra_base_v3_discrim_korquad_aihub_1epoch_retrain.pth')

## Final Open domain question answering

## inference funtion

In [ ]:
stopwords = ['','요새','유독','가끔','틀림없이','때로','필수','최소한','최소','오로지','어차피','다소','이미','만약에','줄곧','꼭','종종','약간','자칫','너무','아예','반드시','비록','한때','만약','무조건',"릴리안느의 폭정에 반기를 들었기 때문이다.하지만 그냥 죽이기엔 그를 따르는 자가 너무","집단 극화를 거쳐 받아들여진 신념은 개인의 정체성을 형성하는 요소가 된다. 때로",
                                                        '어쩔','때때로','왜냐하면','대략','있었','급격히','적어도','점차','만일','언젠가','꾸준히','굳이','이따금','심지어','오직','너무나','감히','어쨌든','절대로','괜찮다고','문득','가끔 (대략','설마','괜찮',"너무 많은걸 알게된 오드리는 횡설수설했고 남자들 역시 별다른 소득이 없자 순순히","왜냐하면 내가 생각",'서서히',
                                                        '최소화하여 수성에 접근하기 위해서는 필수','기꺼이','결코','근데','어느덧','우선','비밀리에','순순히','마땅히','절대',"비록 난폭할지라도 어찌 감히","식민지의 항만 방어를 위해 요새","왜냐하면 내가 생각하는 일을 아주 중지해 버린다면 다음 순간","하루에 토끼 310마리를 잡았으며 꿩, 너구리, 사슴은 그 수를 헤아릴 수 없을 정도",
                                                        "아무래도 좋다\"라고 장자는 말한다. '앎'에는 어떤 확실한 판단은 없으니까, 생각해도 어쩔",'차츰','일부러','쉽사리',"너무 많았고 알렌을 시켜 암살한다.","내가 생각할 동안이다. 왜냐하면","다소 포괄적으로 활용되고 있는데 이를 정의할 때 가장 중요한 요소는 학습자이다. 게임이나 시뮬레이션에는 학습 목적",
                                                        "가덕수로를 따라 올라가야 하는 곳이며 그 곳에 일본군들은 눌차왜성",'아무래도',"너무나 힘든 세상에 도날드 덕은 암소를 죽여 고기를 먹으려고 한다. 그것을 말린 구피와 미키. 어느덧","변경 변화점에 집중되어 이루어져야 한다. 만약","너무 많았고 알렌을 시켜 암살한다. :릴리안느에게는 약혼자 마론왕국","다케다는",
                                                         "주류 기독교 교파들은 메시아 유대교를 기독교의 한 종파로 받아들인다 메시아 유대교의 몇몇 지지자들","대포를 해안포로 활용한 것은 16세기 유럽에서 시작되었다. 유럽 본토와 식민지의 항만 방어를 위해 요새","비록 되어 여러분으로 완성된다는 것을 결코 잊지 않겠습니다. 감사합니다.}} 한번 문 사건은 절대",
                                                         "장소에 있지 않았으나 다른 이들로부터 그 내용을 전해들은 것이다. 곳샬크는, '역사가는 이따금","만약 갑의 가치가 800, 을의 피해는 500일 때도 코스의 정리는 성립한다. 비록","대중들의 독재인가?”라는 하나의 문제 제기는 이미","비록 당장은 진나라의 세력이 강해 점령지의 군민들을 힘으로 누르고 있어 감히 반기를 들지 못하지만, 만일",
                                                         "레온하르트가 자주 릴리안느의 폭정에 반기를 들었기 때문이다.하지만 그냥 죽이기엔 그를 따르는 자가 너무","이따금 소문에 의한 증거를 사용할 수 있음'을 지적한다. 어쨌거나, 2차 목격자","점차 약화하였으며, '닉슨 독트린' 이후로 급격히","학습 게임과 시뮬레이션이라는 용어는 다소","괜찮다고 말했다. 행인은 괜찮지 않다고",
                                                         "최소한 500이상을 받으면 되기 때문에, 500~ 800 사이에서 갑은 이를 기꺼이","오직 십자가의 무조건","너무 많았고 알렌을 시켜 암살한다. :릴리안느","항만 방어를 위해 요새",'대뜸','너무 많았고 알렌','곧잘',"내부 비평이라 한다. R. J. 샤퍼(Shafer)는 '외부 비평은 이따금","방어를 위해 요새","때로는 서로 경쟁하고, 때로",
                                                         "비록 되어 여러분으로 완성된다는 것을 결코","유럽 본토와 식민지의 항만 방어를 위해 요새","생각할 동안이다. 왜냐하면","휘발성 물질은 무엇인가 학습 게임과 시뮬레이션이라는 용어는 다소","각본가로서 먹고 살기 어려울 것이라는 생각 때문에, 꾸준히",'베다는','감각기에',"너무 많았고 알렌을 시켜 암살한다. :릴리안느에게는 약혼자 마론왕국의 젊은 왕 카일 마론"
                                                        ]

In [ ]:
#passage dense embedding
def document_embedding(p_encoder,corpus,tokenizer):
    
    with torch.no_grad():
        p_encoder.eval()
        
        p_embs=[]

        for p in notebook.tqdm(corpus):
            p_seqs = tokenizer(p,padding='max_length',truncation=True, return_tensors='pt').to('cuda')
            p_emb = p_encoder(**p_seqs).to('cpu').numpy()
            del p_seqs
            torch.cuda.empty_cache()
            p_embs.append(p_emb)
            
    p_embs = torch.Tensor(p_embs).squeeze()
    
    return p_embs

In [ ]:
#define get test dataset function

def prepare_test_dataset(test_dataset,contexts,query_passage_score,rank):
    
    context_list = []
    score_list = []
    id_score_list = []

    for _,context_id,score in notebook.tqdm(query_passage_score):
        context_list.append(contexts[context_id[rank]])
        score_list.append(score[rank])
    
    test_dataset = test_dataset['validation']
    test_id_list = test_dataset['id']
    test_query_list = test_dataset['question']
    
    
    for a,b in zip(test_id_list,score_list):
        id_score_list.append((a,b))
        
    
    test_dict = {'id':test_id_list,'question':test_query_list,'context':context_list}
    
    f = Features({'context': Value(dtype='string', id=None),
                      'id': Value(dtype='string', id=None),
                      'question': Value(dtype='string', id=None)})
    
    test_data = DatasetDict({'test': Dataset.from_dict(test_dict, features=f)})
    
    raw_test_dataset = test_data['test']
    
    pad_on_right = mrc_tokenizer.padding_side == "right"

    test_dataset = raw_test_dataset.map(
        prepare_test_features,
        batched=True,
        remove_columns=raw_test_dataset.column_names
    )
    
    return raw_test_dataset,test_dataset,id_score_list

In [ ]:
def prepare_test_dataset_multiple(test_dataset,contexts,query_passage_score):
    
    context_list = []

    for _,context_id,_ in notebook.tqdm(query_passage_score):
        
        context = ''
        
        for idx in context_id:
            context = context + ' ' + contexts[idx]
        
        context_list.append(context)
    
    test_dataset = test_dataset['validation']
    test_id_list = test_dataset['id']
    test_query_list = test_dataset['question']
        
    
    test_dict = {'id':test_id_list,'question':test_query_list,'context':context_list}
    
    f = Features({'context': Value(dtype='string', id=None),
                      'id': Value(dtype='string', id=None),
                      'question': Value(dtype='string', id=None)})
    
    test_data = DatasetDict({'test': Dataset.from_dict(test_dict, features=f)})
    
    raw_test_dataset = test_data['test']
    
    pad_on_right = mrc_tokenizer.padding_side == "right"

    test_dataset = raw_test_dataset.map(
        prepare_test_features,
        batched=True,
        remove_columns=raw_test_dataset.column_names
    )
    
    return raw_test_dataset,test_dataset

In [ ]:
def find_retrieve_score(example_id,id_score_list):
    
    for ex_id,score in id_score_list:
        if example_id == ex_id:
            return score

In [ ]:
#define get_relevant_document function

def get_relevant_doc(q_encoder,query_list,p_embs,k=1): 
    
    with torch.no_grad():
        q_encoder.eval()
        
        q_seqs_list = dense_tokenizer(query_list,padding='max_length',truncation=True, return_tensors='pt').to('cuda')
        
        q_embs = q_encoder(**q_seqs_list).to('cpu')
        
        del q_seqs_list
        torch.cuda.empty_cache()
        
        query_passage_score = []
        
        for i,q_emb in notebook.tqdm(enumerate(q_embs)):
    
            dot_product_scores = torch.matmul(q_emb,torch.transpose(p_embs,0,1)) 
            rank,score= torch.sort(dot_product_scores,descending=True).indices,torch.sort(dot_product_scores,descending=True).values
            query_passage_score.append((query_list[i],rank[:k],score[:k]))

    
    return query_passage_score

In [ ]:
def get_relevant_dense_doc_from_sparse(q_encoder,query_list,p_embs,query_passage_score_sparse,k=1): 
    
    with torch.no_grad():
        q_encoder.eval()
        
        q_seqs_list = dense_tokenizer(query_list,padding='max_length',truncation=True, return_tensors='pt').to('cuda')
        
        q_embs = q_encoder(**q_seqs_list).to('cpu')
        
        del q_seqs_list
        torch.cuda.empty_cache()
        
        query_passage_score = []
        
        for i,a in notebook.tqdm(enumerate(query_passage_score_sparse)):
            
            query,doc_id,doc_score = a
    
            dot_product_scores = torch.matmul(q_embs[i],torch.transpose(p_embs[doc_id],0,1)) 
            rank,score= torch.sort(dot_product_scores,descending=True).indices,torch.sort(dot_product_scores,descending=True).values
            query_passage_score.append((query,doc_id[rank[:k]],score[:k]))

    
    return query_passage_score

In [ ]:
#define get_relevant_document_sparse function

def get_relevant_doc_sparse(sp_matrix, vectorizer, query_list,k=1): 
    
    with torch.no_grad():
        
        query_passage_score = []
        
        for query in notebook.tqdm(query_list):
            
            query_vec= vectorizer.transform([query])
            
            result = query_vec * sp_matrix.T
            
            sorted_result = np.argsort(-result.data)
            doc_scores = result.data[sorted_result]
            doc_ids = result.indices[sorted_result]
            

            query_passage_score.append((query,doc_ids[:k],doc_scores[:k]))

    
    return query_passage_score

In [ ]:
#define get_relevant_document_concat function

def get_relevant_doc_concat(q_encoder,p_embs,sp_matrix,vectorizer,query_list,k=1):
    
    with torch.no_grad():
        
        q_encoder.eval()
        
        q_seqs_list = dense_tokenizer(query_list,padding='max_length',truncation=True, return_tensors='pt').to('cuda')
        
        q_embs = q_encoder(**q_seqs_list).to('cpu')
        
        del q_seqs_list
        torch.cuda.empty_cache()
        
        query_passage_score = []
        
        for q_emb,query in notebook.tqdm(zip(q_embs,query_list)):
    
            dot_product_scores_dense = torch.matmul(q_emb,torch.transpose(p_embs,0,1))
            
            query_vec= vectorizer.transform([query])
            
            dot_product_scores_sparse = torch.tensor((query_vec * sp_matrix.T).toarray())
            
            #dot_product_scores = (dot_product_scores_dense/sum(dot_product_scores_dense)) + (dot_product_scores_sparse.squeeze()/sum(dot_product_scores_sparse.squeeze()))
            
            dot_product_zscores_dense = ((dot_product_scores_dense-torch.mean(dot_product_scores_dense))/torch.std(dot_product_scores_dense))
            
            dot_product_zscores_sparse = ((dot_product_scores_sparse.squeeze()-torch.mean(dot_product_scores_sparse.squeeze()))/torch.std(dot_product_scores_sparse.squeeze()))
            
            dot_product_scores = dot_product_zscores_dense+dot_product_zscores_sparse
            
            rank,score= torch.sort(dot_product_scores,descending=True).indices,torch.sort(dot_product_scores,descending=True).values
            
            query_passage_score.append((query,rank[:k],score[:k]))
            
    
    return query_passage_score
        

In [ ]:
#define get_answer_from_context function

def get_answer_from_context(context, test_dataset, id_score_list, model):
    
    example_id = test_dataset['example_id']
    
    retrieve_score = find_retrieve_score(example_id,id_score_list)
    
    with torch.no_grad():
        
        model.eval()
        

        inputs = {
            'input_ids': torch.tensor([test_dataset['input_ids']], dtype=torch.long).to('cuda'),
            'attention_mask': torch.tensor([test_dataset['attention_mask']], dtype=torch.long).to('cuda'),
            'token_type_ids': torch.tensor([test_dataset['token_type_ids']], dtype=torch.long).to('cuda')
        }

        outputs = model(**inputs)

        start_logits = outputs.start_logits[0].cpu().numpy()
        softmax_start = F.softmax(torch.tensor(start_logits),dim=0).cpu().numpy()
        end_logits = outputs.end_logits[0].cpu().numpy()
        softmax_end = F.softmax(torch.tensor(end_logits),dim=0).cpu().numpy()
        
        offset_mapping = test_dataset["offset_mapping"]


        # Gather the indices the best start/end logits:
        start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
        end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
        

        valid_answers = []
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                # to part of the input_ids that are not in the context.
                if (
                    start_index >= len(offset_mapping)
                    or end_index >= len(offset_mapping)
                    or offset_mapping[start_index] is None
                    or offset_mapping[end_index] is None
                ):
                    continue
                # Don't consider answers with a length that is either < 0 or > max_answer_length.
                if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                    continue
                if start_index <= end_index: # We need to refine that test to check the answer is inside the context
                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    
                    if context[start_char: end_char] in stopwords:
                        pass
                    elif '[UNK]' in context[start_char: end_char]:
                        pass
                    else:
                        valid_answers.append(
                            {
                                #"score": start_logits[start_index] + end_logits[end_index],
                                'score': softmax_start[start_index] * softmax_end[end_index] + retrieve_score,
                                "text": context[start_char: end_char],
                                'id' : example_id
                            }
                        )

        #valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0, 'id':example_id}

        
        
    return best_answer
    #return valid_answers

In [ ]:
def feature_per_example(raw_test_dataset,test_dataset):
    
    example_id_to_index = {k: i for i, k in enumerate(raw_test_dataset["id"])}
    features_per_example = collections.defaultdict(list)
    
    for i, feature in enumerate(notebook.tqdm(test_dataset)):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)
    
    return features_per_example

In [ ]:
def get_answer_from_multiple_context(context, test_dataset, model):
    
    example_id = test_dataset['example_id']
    
    with torch.no_grad():
        
        model.eval()
        

        inputs = {
            'input_ids': torch.tensor([test_dataset['input_ids']], dtype=torch.long).to('cuda'),
            'attention_mask': torch.tensor([test_dataset['attention_mask']], dtype=torch.long).to('cuda'),
            'token_type_ids': torch.tensor([test_dataset['token_type_ids']], dtype=torch.long).to('cuda')
        }

        outputs = model(**inputs)

        start_logits = outputs.start_logits[0].cpu().numpy()
        softmax_start = F.softmax(torch.tensor(start_logits),dim=0).cpu().numpy()
        end_logits = outputs.end_logits[0].cpu().numpy()
        softmax_end = F.softmax(torch.tensor(end_logits),dim=0).cpu().numpy()
        
        offset_mapping = test_dataset["offset_mapping"]
        
        # Gather the indices the best start/end logits:
        start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
        end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
        

        valid_answers = []
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                # to part of the input_ids that are not in the context.
                if (
                    start_index >= len(offset_mapping)
                    or end_index >= len(offset_mapping)
                    or offset_mapping[start_index] is None
                    or offset_mapping[end_index] is None
                ):
                    continue
                # Don't consider answers with a length that is either < 0 or > max_answer_length.
                if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                    continue
                if start_index <= end_index: # We need to refine that test to check the answer is inside the context
                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    
                    if context[start_char: end_char] in stopwords:
                        pass
                    elif '[UNK]' in context[start_char: end_char]:
                        pass
                    else:
                        valid_answers.append(
                            {
                                #"score": start_logits[start_index] + end_logits[end_index],
                                'score': softmax_start[start_index] + softmax_end[end_index],
                                "text": context[start_char: end_char],
                                'id' : example_id
                            }
                        )

        #valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0, 'id':example_id}

        
        
    return best_answer
    #return valid_answers

In [ ]:
#define ensemble three function

def get_answer_from_context_ensemble_five(context,test_dataset, id_score_list, model1, model2, model3,model4,model5):
    
    example_id = test_dataset['example_id']
    
    retrieve_score = find_retrieve_score(example_id,id_score_list)
    
    with torch.no_grad():
        
        model1.eval()
        model2.eval()
        model3.eval()
        model4.eval()
        model5.eval()
        

        inputs = {
            'input_ids': torch.tensor([test_dataset['input_ids']], dtype=torch.long).to('cuda'),
            'attention_mask': torch.tensor([test_dataset['attention_mask']], dtype=torch.long).to('cuda'),
            'token_type_ids': torch.tensor([test_dataset['token_type_ids']], dtype=torch.long).to('cuda')
        }

        outputs1 = model1(**inputs)
        outputs2 = model2(**inputs)
        outputs3 = model3(**inputs)
        outputs4 = model4(**inputs)
        outputs5 = model5(**inputs)

        start_logits1 = outputs1.start_logits[0].cpu().numpy()
        softmax_start1 = F.softmax(torch.tensor(start_logits1),dim=0).cpu().numpy()
        end_logits1 = outputs1.end_logits[0].cpu().numpy()
        softmax_end1 = F.softmax(torch.tensor(end_logits1),dim=0).cpu().numpy()
        
        start_logits2 = outputs2.start_logits[0].cpu().numpy()
        softmax_start2 = F.softmax(torch.tensor(start_logits2),dim=0).cpu().numpy()
        end_logits2 = outputs2.end_logits[0].cpu().numpy()
        softmax_end2 = F.softmax(torch.tensor(end_logits2),dim=0).cpu().numpy()
        
        start_logits3 = outputs3.start_logits[0].cpu().numpy()
        softmax_start3 = F.softmax(torch.tensor(start_logits3),dim=0).cpu().numpy()
        end_logits3 = outputs3.end_logits[0].cpu().numpy()
        softmax_end3 = F.softmax(torch.tensor(end_logits3),dim=0).cpu().numpy()
        
        start_logits4 = outputs4.start_logits[0].cpu().numpy()
        softmax_start4 = F.softmax(torch.tensor(start_logits4),dim=0).cpu().numpy()
        end_logits4 = outputs4.end_logits[0].cpu().numpy()
        softmax_end4 = F.softmax(torch.tensor(end_logits4),dim=0).cpu().numpy()
        
        start_logits5 = outputs5.start_logits[0].cpu().numpy()
        softmax_start5 = F.softmax(torch.tensor(start_logits5),dim=0).cpu().numpy()
        end_logits5 = outputs5.end_logits[0].cpu().numpy()
        softmax_end5 = F.softmax(torch.tensor(end_logits5),dim=0).cpu().numpy()
        
        
        softmax_start = (softmax_start1+softmax_start2+softmax_start3+softmax_start4+softmax_start5)/5
        softmax_end = (softmax_end1+softmax_end2+softmax_end3+softmax_end4+softmax_end5)/5
        
        
        offset_mapping = test_dataset["offset_mapping"]


        # Gather the indices the best start/end logits:
        start_indexes = np.argsort(softmax_start)[-1 : -n_best_size - 1 : -1].tolist()
        end_indexes = np.argsort(softmax_end)[-1 : -n_best_size - 1 : -1].tolist()
        

        valid_answers = []
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                # to part of the input_ids that are not in the context.
                if (
                    start_index >= len(offset_mapping)
                    or end_index >= len(offset_mapping)
                    or offset_mapping[start_index] is None
                    or offset_mapping[end_index] is None
                ):
                    continue
                # Don't consider answers with a length that is either < 0 or > max_answer_length.
                if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                    continue
                if start_index <= end_index: # We need to refine that test to check the answer is inside the context
                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    
                    if context[start_char: end_char] in stopwords:
                        pass
                    elif '[UNK]' in context[start_char: end_char]:
                        pass
                    else:
                        valid_answers.append(
                            {
                                #"score": start_logits[start_index] + end_logits[end_index],
                                'score': softmax_start[start_index] * softmax_end[end_index],# retrieve_score,
                                "text": context[start_char: end_char],
                                'id' : example_id
                            }
                        )

        #valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0, 'id':example_id}

        
        
    return best_answer
    #return valid_answers

In [ ]:
def get_answer_from_multiple_context_ensemble_five(context,
                                                    test_dataset, model1, model2, model3, model4):
    
    example_id = test_dataset['example_id']
    
    with torch.no_grad():
        
        model1.eval()
        model2.eval()
        model3.eval()
        model4.eval()
        #model5.eval()
        

        inputs = {
            'input_ids': torch.tensor([test_dataset['input_ids']], dtype=torch.long).to('cuda'),
            'attention_mask': torch.tensor([test_dataset['attention_mask']], dtype=torch.long).to('cuda'),
            'token_type_ids': torch.tensor([test_dataset['token_type_ids']], dtype=torch.long).to('cuda')
        }

        outputs1 = model1(**inputs)
        outputs2 = model2(**inputs)
        outputs3 = model3(**inputs)
        outputs4 = model4(**inputs)
        #outputs5 = model5(**inputs)

        start_logits1 = outputs1.start_logits[0].cpu().numpy()
        softmax_start1 = F.softmax(torch.tensor(start_logits1),dim=0).cpu().numpy()
        end_logits1 = outputs1.end_logits[0].cpu().numpy()
        softmax_end1 = F.softmax(torch.tensor(end_logits1),dim=0).cpu().numpy()
        
        start_logits2 = outputs2.start_logits[0].cpu().numpy()
        softmax_start2 = F.softmax(torch.tensor(start_logits2),dim=0).cpu().numpy()
        end_logits2 = outputs2.end_logits[0].cpu().numpy()
        softmax_end2 = F.softmax(torch.tensor(end_logits2),dim=0).cpu().numpy()
        
        start_logits3 = outputs3.start_logits[0].cpu().numpy()
        softmax_start3 = F.softmax(torch.tensor(start_logits3),dim=0).cpu().numpy()
        end_logits3 = outputs3.end_logits[0].cpu().numpy()
        softmax_end3 = F.softmax(torch.tensor(end_logits3),dim=0).cpu().numpy()
        
        start_logits4 = outputs4.start_logits[0].cpu().numpy()
        softmax_start4 = F.softmax(torch.tensor(start_logits4),dim=0).cpu().numpy()
        end_logits4 = outputs4.end_logits[0].cpu().numpy()
        softmax_end4 = F.softmax(torch.tensor(end_logits4),dim=0).cpu().numpy()
        
        #start_logits5 = outputs5.start_logits[0].cpu().numpy()
        #softmax_start5 = F.softmax(torch.tensor(start_logits5),dim=0).cpu().numpy()
        #end_logits5 = outputs5.end_logits[0].cpu().numpy()
        #softmax_end5 = F.softmax(torch.tensor(end_logits5),dim=0).cpu().numpy()
        
        
        softmax_start = (softmax_start1+softmax_start2+softmax_start3+softmax_start4)/4
        softmax_end = (softmax_end1+softmax_end2+softmax_end3+softmax_end4)/4
        
        
        offset_mapping = test_dataset["offset_mapping"]

        # Gather the indices the best start/end logits:
        start_indexes = np.argsort(softmax_start)[-1 : -n_best_size - 1 : -1].tolist()
        end_indexes = np.argsort(softmax_end)[-1 : -n_best_size - 1 : -1].tolist()
        

        valid_answers = []
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                # to part of the input_ids that are not in the context.
                if (
                    start_index >= len(offset_mapping)
                    or end_index >= len(offset_mapping)
                    or offset_mapping[start_index] is None
                    or offset_mapping[end_index] is None
                ):
                    continue
                # Don't consider answers with a length that is either < 0 or > max_answer_length.
                if (end_index < start_index) or (end_index - start_index + 1 > max_answer_length):
                    continue
                if start_index <= end_index: # We need to refine that test to check the answer is inside the context
                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    
                    if context[start_char: end_char] in stopwords:
                        pass
                    elif '[UNK]' in context[start_char: end_char]:
                        pass
                        
                    #elif len(context[start_char: end_char]) > 35:
                        #continue
                    else:
                        valid_answers.append(
                            {
                                #"score": start_logits[start_index] + end_logits[end_index],
                                'score': softmax_start[start_index] + softmax_end[end_index],
                                "text": context[start_char: end_char],
                                'id' : example_id
                            }
                        )

        #valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0, 'id':example_id}

        
        
    return best_answer
    #return valid_answers

In [ ]:
def prepare_test_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = mrc_tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
def write_prediction_dict(raw_test_dataset,test_dataset,id_score_list,features_per_example):
    
    prediction_dict = {}
    
    for example_index, _ in enumerate(notebook.tqdm(raw_test_dataset)):

        feature_indices = features_per_example[example_index]
        
        for index in feature_indices:

            answer = get_answer_from_context(raw_test_dataset[example_index]['context'], test_dataset[index], id_score_list, mrc_model)
        
            try:

                 if len(prediction_dict[answer['id']])==2:

                    if answer['score'] > prediction_dict[answer['id']][1]:

                        prediction_dict[answer['id']] = (answer['text'],answer['score'])

            except KeyError:

                prediction_dict[answer['id']] = (answer['text'],answer['score'])

                
    
    return prediction_dict

In [ ]:
def write_prediction_dict_multiple(raw_test_dataset, test_dataset, features_per_example):
    
    prediction_dict = {}
        
    for example_index, _ in enumerate(notebook.tqdm(raw_test_dataset)):

        feature_indices = features_per_example[example_index]
        
        for index in feature_indices:

            answer = get_answer_from_multiple_context(raw_test_dataset[example_index]['context'], test_dataset[index], mrc_model)
        
            try:

                 if len(prediction_dict[answer['id']])==2:

                    if answer['score'] > prediction_dict[answer['id']][1]:

                        prediction_dict[answer['id']] = (answer['text'],answer['score'])

            except KeyError:

                prediction_dict[answer['id']] = (answer['text'],answer['score'])

                
    
    return prediction_dict

In [ ]:
def write_prediction_dict_ensemble_five(raw_test_dataset, test_dataset, id_score_list, features_per_example):
    
    prediction_dict = {}

    for example_index, _ in enumerate(notebook.tqdm(raw_test_dataset)):

        feature_indices = features_per_example[example_index]

        for index in feature_indices:

            answer = get_answer_from_context_ensemble_five(raw_test_dataset[example_index]['context'], test_dataset[index], id_score_list, mrc_model1,mrc_model2,mrc_model3,
                                                           mrc_model4,mrc_model5)

            try:

                 if len(prediction_dict[answer['id']])==2:

                    if answer['score'] > prediction_dict[answer['id']][1]:

                        prediction_dict[answer['id']] = (answer['text'],answer['score'])

            except KeyError:

                prediction_dict[answer['id']] = (answer['text'],answer['score'])

                
    
    return prediction_dict

In [ ]:
def write_prediction_dict_ensemble_five_multiple(raw_test_dataset, test_dataset, features_per_example):
    
    prediction_dict = {}
    
    for example_index, _ in enumerate(notebook.tqdm(raw_test_dataset)):

        feature_indices = features_per_example[example_index]
        
        for index in feature_indices:

            answer = get_answer_from_multiple_context_ensemble_five(raw_test_dataset[example_index]['context'], test_dataset[index], mrc_model1, mrc_model2, mrc_model3,
                                                                    mrc_model4)
            try:

                 if len(prediction_dict[answer['id']])==2:

                    if answer['score'] > prediction_dict[answer['id']][1]:

                        prediction_dict[answer['id']] = (answer['text'],answer['score'])

            except KeyError:

                prediction_dict[answer['id']] = (answer['text'],answer['score'])

                
    
    return prediction_dict

In [ ]:
def rewrite_prediction_dict(prediction_dict,raw_test_dataset,test_dataset,id_score_list,features_per_example):
    
    for example_index, _ in enumerate(notebook.tqdm(raw_test_dataset)):

        feature_indices = features_per_example[example_index]
        
        for index in feature_indices:

            answer = get_answer_from_context(raw_test_dataset[example_index]['context'], test_dataset[index], id_score_list, mrc_model)
        

            if answer['score'] > prediction_dict[answer['id']][1]:

                prediction_dict[answer['id']] = (answer['text'],answer['score'])
                
    return prediction_dict

In [ ]:
def rewrite_prediction_dict_ensemble_five(prediction_dict,raw_test_dataset,test_dataset,id_score_list,features_per_example):
    
    for example_index, _ in enumerate(notebook.tqdm(raw_test_dataset)):

        feature_indices = features_per_example[example_index]
        
        for index in feature_indices:

            answer = get_answer_from_context_ensemble_five(raw_test_dataset[example_index]['context'], test_dataset[index], id_score_list, mrc_model1,mrc_model2,mrc_model3,
                                                           mrc_model4)
        
            

            if answer['score'] > prediction_dict[answer['id']][1]:

                prediction_dict[answer['id']] = (answer['text'],answer['score'])
                
    return prediction_dict

In [ ]:
def final_inference(k,contexts,query_passage_score):
    
    for rank in range(k):

        if rank == 0:

            test_dataset = load_from_disk('input/data/data/test_dataset')

            raw_test_dataset,test_dataset,id_score_list = prepare_test_dataset(test_dataset,contexts,query_passage_score,rank)
            
            features_per_example = feature_per_example(raw_test_dataset,test_dataset)

            prediction_dict = write_prediction_dict(raw_test_dataset,test_dataset,id_score_list,features_per_example)
            

        else:

            test_dataset = load_from_disk('input/data/data/test_dataset')

            raw_test_dataset,test_dataset,id_score_list = prepare_test_dataset(test_dataset,contexts,query_passage_score,rank)
            
            features_per_example = feature_per_example(raw_test_dataset,test_dataset)

            prediction_dict = rewrite_prediction_dict(prediction_dict,raw_test_dataset,test_dataset,id_score_list,features_per_example)

    return prediction_dict

In [ ]:
def final_inference_multiple(contexts,query_passage_score):

    test_dataset = load_from_disk('input/data/data/test_dataset')

    raw_test_dataset, test_dataset = prepare_test_dataset_multiple(test_dataset,contexts,query_passage_score)

    features_per_example = feature_per_example(raw_test_dataset,test_dataset)
    
    prediction_dict = write_prediction_dict_multiple(raw_test_dataset,test_dataset,features_per_example)

    return prediction_dict

In [ ]:
def final_inference_ensemble_five(k,contexts,query_passage_score):
    
    for rank in range(k):

        if rank == 0:

            test_dataset = load_from_disk('input/data/data/test_dataset')

            raw_test_dataset,test_dataset,id_score_list = prepare_test_dataset(test_dataset,contexts,query_passage_score,rank)
            
            features_per_example = feature_per_example(raw_test_dataset,test_dataset)

            prediction_dict = write_prediction_dict_ensemble_five(raw_test_dataset,test_dataset,id_score_list,features_per_example)
            

        else:

            test_dataset = load_from_disk('input/data/data/test_dataset')

            raw_test_dataset,test_dataset,id_score_list = prepare_test_dataset(test_dataset,contexts,query_passage_score,rank)
            
            features_per_example = feature_per_example(raw_test_dataset,test_dataset)

            prediction_dict = rewrite_prediction_dict_ensemble_five(prediction_dict,raw_test_dataset,test_dataset,id_score_list,features_per_example)

    return prediction_dict

In [ ]:
def final_inference_ensemble_five_multiple(contexts,query_passage_score):
    
    test_dataset = load_from_disk('input/data/data/test_dataset')

    raw_test_dataset, test_dataset = prepare_test_dataset_multiple(test_dataset,contexts,query_passage_score)

    features_per_example = feature_per_example(raw_test_dataset,test_dataset)
    
    prediction_dict = write_prediction_dict_ensemble_five_multiple(raw_test_dataset, test_dataset, features_per_example)

    return prediction_dict

## prepare question

In [ ]:
test_dataset = load_from_disk('input/data/data/test_dataset')

In [ ]:
test_dataset

DatasetDict({
    validation: Dataset({
        features: ['id', 'question'],
        num_rows: 600
    })
})

In [ ]:
test_dataset = test_dataset['validation']

In [ ]:
test_query_list = test_dataset['question']

In [ ]:
test_id_list = test_dataset['id']

## prepare context

In [ ]:
with open('input/data/data/wikipedia_documents.json', "r") as f:
    wiki = json.load(f)

    contexts = list(dict.fromkeys([v['text'] for v in wiki.values()])) # set 은 매번 순서가 바뀌므로
    print(f"Lengths of unique contexts : {len(contexts)}")
    ids = list(range(len(contexts)))

Lengths of unique contexts : 56737


In [ ]:
clean_context_list = contexts

## preprocessing wikipedia context

In [ ]:
def clean_context(data):
    
    #remove special character
    
    pattern = '[-=+#/,\?:^$@*.\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》《\n]' #want to remove pattern
    
    repl = '' #replace pattern
 
    text = re.sub(pattern = pattern, repl=repl, string=data)
    
    text = text.replace('\\n','')
    
    text = text.replace('[','')
    
    text = text.replace(']','')
    
    text = text.replace('〉','')
    
    text = text.replace('〈','')
    
    text = text.replace('｜','')
    
    text = text.replace('。','')
    
    text = text.replace('{','')
    
    text = text.replace('}','')
    
    text = text.replace('·','')
    
    text = text.replace('「','')
    text = text.replace('」','')
    
    text = text.replace('『','')
    
    text = text.replace('–','')
    
    text = text.replace("’",'')
    
    text = text.replace('“','')
    
    text = text.replace('”','')
 
    return text

In [ ]:
def remove_newline(data):
    
    #remove special character
    
    pattern = '[\n]' #want to remove pattern
    
    repl = ' ' #replace pattern
 
    text = re.sub(pattern = pattern, repl=repl, string=data)
    
    text = text.replace('\\n',' ')
    
    return text

In [ ]:
def remove_useless_breacket(texts):
    """
    위키피디아 전처리를 위한 함수입니다.
    괄호 내부에 의미가 없는 정보를 제거합니다.
    아무런 정보를 포함하고 있지 않다면, 괄호를 통채로 제거합니다.
    ``수학(,)`` -> ``수학``
    ``수학(數學,) -> ``수학(數學)``
    """
    bracket_pattern = re.compile(r"\((.*?)\)")
    preprocessed_text = []
    for text in texts:
        modi_text = ""
        text = text.replace("()", "")  # 수학() -> 수학
        brackets = bracket_pattern.search(text)
        if not brackets:
            if text:
                preprocessed_text.append(text)
                continue
        replace_brackets = {}
        # key: 원본 문장에서 고쳐야하는 index, value: 고쳐져야 하는 값
        # e.g. {'2,8': '(數學)','34,37': ''}
        while brackets:
            index_key = str(brackets.start()) + "," + str(brackets.end())
            bracket = text[brackets.start() + 1 : brackets.end() - 1]
            infos = bracket.split(",")
            modi_infos = []
            for info in infos:
                info = info.strip()
                if len(info) > 0:
                    modi_infos.append(info)
            if len(modi_infos) > 0:
                replace_brackets[index_key] = "(" + ", ".join(modi_infos) + ")"
            else:
                replace_brackets[index_key] = ""
            brackets = bracket_pattern.search(text, brackets.start() + 1)
        end_index = 0
        for index_key in replace_brackets.keys():
            start_index = int(index_key.split(",")[0])
            modi_text += text[end_index:start_index]
            modi_text += replace_brackets[index_key]
            end_index = int(index_key.split(",")[1])
        modi_text += text[end_index:]
        modi_text = modi_text.strip()
        if modi_text:
            preprocessed_text.append(modi_text)
    return preprocessed_text

In [ ]:
def remove_repeated_spacing(texts):
    """
    두 개 이상의 연속된 공백을 하나로 치환합니다.
    ``오늘은    날씨가   좋다.`` -> ``오늘은 날씨가 좋다.``
    """
    preprocessed_text = []
    for text in texts:
        text = re.sub(r"\s+", " ", text).strip()
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

## embedding context

In [ ]:
p_embs = document_embedding(p_encoder,clean_context_list,dense_tokenizer)

In [ ]:
p_embs.shape

torch.Size([55963, 768])

## save wiki context embedding

In [ ]:
#save passage embedding

with open('dense_wiki_passage_v4.pkl', 'wb') as f:
    pickle.dump(p_embs, f)

In [ ]:
#load passage embedding

with open('dense_wiki_passage_v4.pkl', 'rb') as f:
    p_embs = pickle.load(f)

# define TF-IDF sparse embedding

# get cleansing wikipedia text

In [ ]:
#define get clean wikipedia context function

def get_clean_wikipedia_text(contexts):
    
    clean_context_list1 = remove_useless_breacket(contexts)
    
    clean_context_list2 = []
    clean_context_list3 = []
    clean_context_list4 = []
    clean_context_list5 = []
    
    
    #remove japan text
    for context in notebook.tqdm(clean_context_list1):
        clean_context_list2.append(remove_language('30A0','30FF',context))

    for context in notebook.tqdm(clean_context_list2):
        clean_context_list3.append(remove_language('3040','309F',context))
        
    
    #remove china text

    for context in notebook.tqdm(clean_context_list3):
        clean_context_list4.append(remove_language('4E00','9FBF',context))
        
    
    #remove special character
        
    for context in notebook.tqdm(clean_context_list4):
        clean_context_list5.append(clean_context(context))
        
    
    #remove repeated space
    clean_context_list = remove_repeated_spacing(clean_context_list5)
    
    return clean_context_list

In [ ]:
def get_clean_wikipedia_text_v2(contexts):
    
    clean_context_list1 = remove_useless_breacket(contexts)
    
    clean_context_list2 = []
        
    #remove special character
        
    for context in notebook.tqdm(clean_context_list1):
        clean_context_list2.append(remove_newline(context))
        
    
    #remove repeated space
    clean_context_list = remove_repeated_spacing(clean_context_list2)
    
    return clean_context_list

In [ ]:
def get_clean_wikipedia_text_v3(contexts):
    
    clean_context_list1 = remove_useless_breacket(contexts)
            
    #remove repeated space
    clean_context_list = remove_repeated_spacing(clean_context_list1)
    
    return clean_context_list

In [ ]:
def get_clean_wikipedia_text_v4(contexts):
    
    clean_context_list1 = []
        
    #remove special character
        
    for context in notebook.tqdm(contexts):
        clean_context_list1.append(remove_newline(context))
    
    #clean_context_list2 = []
    
    #for context in notebook.tqdm(clean_context_list1):
        #context = context.replace('티폰','튀폰')
        #clean_context_list2.append(context)
        
            
    #remove repeated space
    clean_context_list = remove_repeated_spacing(clean_context_list1)
    
    return clean_context_list

In [ ]:
#read wikipedia raw context 

with open('input/data/data/wikipedia_documents.json', "r") as f:
    wiki = json.load(f)

    contexts = list(dict.fromkeys([v['text'] for v in wiki.values()])) # set 은 매번 순서가 바뀌므로
    print(f"Lengths of unique contexts : {len(contexts)}")
    ids = list(range(len(contexts)))

Lengths of unique contexts : 56737


In [ ]:
clean_context_list = get_clean_wikipedia_text_v4(contexts)

In [ ]:
clean_context_list[5397]

'티폰(Τυφών)은 그리스 신화에 등장하는 가장 강하고 무서우며, 엄청나게 거대한 거인이다. 영어의 태풍(Typhoon)의 어원이기도 하다.(태풍의 광둥어 발음인 daaih-fùng이 변형된 말이라고도 한다.) 머리에서 허벅지까지가 인간이었지만, 사람의 머리 대신에 눈에서 번갯불와 불꽃을 내뿜을 수 있는 100개의 용의 머리가 돋아나 있었고, 두 개의 대퇴부에서 밑으로는 똬리를 튼 거대한 뱀의 모습을 지니고 있다. 온 몸을 뒤덮고 있는 깃털과 날개는 항상 그 자신이 일으키는 격렬한 폭풍 때문에 휘날리고 있다. 그의 어깨는 하늘에 닿고, 100개의 머리는 우주에 있는 별을 스치며, 두 팔을 벌리면 세계의 동쪽과 서쪽의 끝까지 닿는다고 한다. 그가 날개를 펼치면 태양빛이 비춰지지 않아 세계가 어둠에 잠식된다고 한다. 또한 산과 땅을 찢고 하늘을 가를 정도로 힘이 세고, 그가 불을 뿜으면 그 어떤것도 흔적이 남지 않았다. 아무리 신들이라 해도 이런 튀폰을 감히 당해낼 이가 없었다 한다. 티폰이 한번 지나간 자리에는 나무들이 부러지고 흙이 파헤쳐지고 파괴되며 그 어떤 것들이라도 소멸되버리거나 혹은 불타버려서 그림자조차 남지 않을 정도라고 한다. 대지의 여신 가이아는 제우스가 크로노스를 물리치고 신들의 지배자 자리에 오르자 이에 분노하여 크로노스의 원수를 갚기 위해 그녀의 또 다른 배우자인 타르타로스와 관계를 맺어 그녀의 마지막 자식인 튀폰을 낳았다. 티폰은 가이아의 아들이라는 가설이 좀 더 확실하다. 일설에서 튀폰은 제우스가 바람을 피운 것에 복수하기 위해 헤라가 크로노스로부터 받은 알에서 태어나 델포이의 큰 뱀 파이톤에 의해 키워졌다고도 한다. 어느 쪽이든 튀폰은 무럭무럭 커 가면서 힘이 생기자 제우스를 물리치기 위해 올림포스 산으로 진군하였다. 그에 두려워한 올림포스 신들은 전부 이집트로 도망갔으나, 그 자리를 지킨 아테네의 비웃음에 참을 수 없던 제우스는 다시 올림포스로 돌아왔다. 그러나 튀폰의 힘에 굴복한 제우스는 그에게 힘줄을 잘려 아무힘도 쓸 수 없었다.

In [ ]:
clean_context_list[5397]=clean_context_list[5397].replace('티폰','튀폰')

In [ ]:
clean_context_list = list(set(clean_context_list))

In [ ]:
len(clean_context_list)

55963

In [ ]:
test_query_list[0] = test_query_list[0].replace("유령'","'유령'")

In [ ]:
test_query_list[0]

"'유령'은 어느 행성에서 지구로 왔는가?"

# define term frequency

In [ ]:
#define compute raw count term frequency in one passage

def compute_tf_score(context,vocab):
    
    tf_passage = []
    
    tokenized_passage = sparse_tokenizer.morphs(context)
    
    for term in vocab.keys():
        tf_passage.append(tokenized_passage.count(term))
        
    
    return tf_passage

In [ ]:
tf_context_passage = compute_tf_score(clean_context_list[0],vocab)

In [ ]:
len(tf_context_passage)

292340

In [ ]:
print(tf_context_passage[:100])

[8, 2, 9, 5, 5, 1, 1, 1, 1, 3, 5, 1, 1, 2, 2, 3, 3, 5, 1, 1, 1, 3, 9, 7, 9, 9, 5, 1, 1, 1, 3, 3, 1, 1, 2, 1, 2, 2, 1, 1, 1, 7, 2, 1, 1, 1, 3, 1, 5, 1, 1, 3, 3, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2]


In [ ]:
def compute_log_tf_score(context,vocab):
    
    sparse_emb = []
    
    tf_passage = compute_tf_score(context,vocab)

    for raw_count in tf_passage:
        sparse_emb.append(np.log(1+raw_count))

    sparse_emb = torch.tensor(sparse_emb)
    
    return sparse_emb
        

In [ ]:
tf_context_passage = compute_log_tf_score(clean_context_list[0],vocab)

In [ ]:
tf_context_passage[:100]

tensor([2.1972, 1.0986, 2.3026, 1.7918, 1.7918, 0.6931, 0.6931, 0.6931, 0.6931,
        1.3863, 1.7918, 0.6931, 0.6931, 1.0986, 1.0986, 1.3863, 1.3863, 1.7918,
        0.6931, 0.6931, 0.6931, 1.3863, 2.3026, 2.0794, 2.3026, 2.3026, 1.7918,
        0.6931, 0.6931, 0.6931, 1.3863, 1.3863, 0.6931, 0.6931, 1.0986, 0.6931,
        1.0986, 1.0986, 0.6931, 0.6931, 0.6931, 2.0794, 1.0986, 0.6931, 0.6931,
        0.6931, 1.3863, 0.6931, 1.7918, 0.6931, 0.6931, 1.3863, 1.3863, 1.0986,
        1.0986, 1.0986, 1.0986, 1.0986, 0.6931, 1.0986, 0.6931, 0.6931, 0.6931,
        0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931,
        1.0986, 0.6931, 0.6931, 1.6094, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931,
        0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 0.6931,
        0.6931, 0.6931, 0.6931, 0.6931, 0.6931, 1.0986, 1.0986, 0.6931, 0.6931,
        1.0986], dtype=torch.float64)

# define inverse document frequency

In [ ]:
def document_frequency(contexts,term):
    
    df_score = 0
    
    for context in contexts:
        
        #document = sparse_tokenizer.morphs(context)
        
        if term in context:
            
            df_score += 1
    
    return df_score

In [ ]:
def compute_log_idf_score(contexts,vocab):
    
    sparse_emb = []
    
    total_document = len(contexts)
    
    for term in notebook.tqdm(vocab.keys()):
        
        df_score_term = document_frequency(contexts,term)
        
        sparse_emb.append(np.log(total_document/(1+df_score_term)))
        
    sparse_emb = torch.tensor(sparse_emb)
    
    return sparse_emb

In [ ]:
sparse_emb = compute_log_idf_score(clean_context_list,vocab)

# sparse retrieval using TF-IDF vectorizer

In [ ]:
sparse_tokenizer = Mecab()

def mecab_tokenize(text):
    
    return sparse_tokenizer.morphs(text)

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=mecab_tokenize, ngram_range=(1,2),sublinear_tf=True)

In [ ]:
vectorizer.fit(clean_context_list)

sp_matrix = vectorizer.transform(clean_context_list)

/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
sp_matrix.shape

(56737, 3896368)

In [ ]:
df = pd.DataFrame(sp_matrix[0].T.todense(), index=vectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(df.head(10))

         TF-IDF
널리 받   0.147180
개 나라   0.146110
인 승인   0.135684
나열 하   0.134132
째 부분   0.126279
를 나열   0.124468
이 목록   0.117880
나열     0.115657
목록     0.111317
다고 여기  0.109008


In [ ]:
query_vec = vectorizer.transform([test_query_list[0]])

In [ ]:
result = query_vec * sp_matrix.T
result.shape

(1, 56737)

In [ ]:
result.data.shape

(55562,)

In [ ]:
torch.tensor(result.toarray()).shape

torch.Size([1, 56737])

In [ ]:
sorted_result = np.argsort(-result.data)
doc_scores = result.data[sorted_result]
doc_ids = result.indices[sorted_result]

In [ ]:
k=10
doc_scores[:k], doc_ids[:k]

(array([0.07068501, 0.06176537, 0.055114  , 0.05395561, 0.05392631,
        0.0538449 , 0.05332264, 0.05276092, 0.05193748, 0.05108912]),
 array([45971, 17229, 53819, 22606, 38423, 41094, 11868, 24508, 38450,
        39470], dtype=int32))

In [ ]:
print("[Search query]\n", test_query_list[0], "\n")

for i in range(k):
    print("Top-%d passage with score %.4f" % (i + 1, doc_scores[i]))
    doc_id = doc_ids[i]
    print(clean_context_list[doc_id], "\n") #set을 한 이유가 여기 있었네 중복이 있어서 

[Search query]
 유령'은 어느 행성에서 지구로 왔는가? 

Top-1 passage with score 0.0707
The Impossible Planet KBS 방영 제목 블랙홀의 저주는 영국의 SF 드라마 닥터 후 시리즈 2의 여덟 번째 에피소드이다 2006년 6월 3일 영국에서 처음 방송되었으며 The Satan Pit과 함께 2부작으로 된 에피소드 중 전편에 해당된다 줄거리는 타디스가 우연히 어느 행성 위에 건설된 기지 내부에 착륙하는 것부터 시작된다 그 행성은 블랙홀 주변을 공전하고 있었는데 블랙홀 속으로 빨려들어가지 않는다는 것은 닥터도 어리둥절할 정도로 불가사의한 상황이었다 그 기지에서는 행성의 공전을 가능케 하는 알수없는 에너지 근원지를 찾아 지하로 파내려가는 작업을 하고 있었는데 인류 제국에 이익이 되지 않을까 싶어서였다 그러나 그곳에는 먼 옛날의 악마가 살고 있었다는 것이 밝혀지고 마침내 깨어나고 만다 

Top-2 passage with score 0.0618
HD 40307 b의 질량은 적어도 지구의 42배 수준인데 기존 가스 행성에 비하면 매우 가벼우며 슈퍼지구로 분류된다 이 행성은 어머니 항성 HD 40307을 43일에 1회 공전하며 항성으로부터의 거리는 0047 천문단위 정도이다 행성의 공전궤도 이심률은 0에 가까운데 이는 행성이 항성으로부터 원형에 가까운 궤도를 유지한다는 의미이다HD 40307의 중원소 함량은 다른 외계 행성을 거느린 항성들에 비해 낮은 편이다 이 사실은 “항성이 태어날 때의 중원소 함량이 자신의 강착 원반으로부터 가스 행성이나 암석 행성이 탄생할지 아닐지 여부를 결정짓는다”라는 가설을 지지한다 

Top-3 passage with score 0.0551
니스 모형Nice model Nice은 태양계의 역학적 진화에 대한 하나의 모형으로 2005년 모형이 처음 개발된 코트다쥐르 천문대의 위치인 프랑스의 도시 니스의 이름을 땄다 니스 모형은 목성형 행성이 원시 행성계 원반의 소멸 이후 기존의 밀집 분포에

## get relevant document using bm25 score

In [ ]:
sparse_tokenizer = Mecab()

def mecab_tokenize(text):
    
    return sparse_tokenizer.morphs(text)

In [ ]:
def get_relevant_doc_bm25(clean_context_list,test_query_list,k):
    
    tokenized_context = [mecab_tokenize(context) for context in notebook.tqdm(clean_context_list)]
    
    bm25 = BM25Okapi(tokenized_context)
    
    query_passage_score = []
        
    for query in notebook.tqdm(test_query_list):
        
        tokenized_query = mecab_tokenize(query)
        
        scores = torch.tensor(bm25.get_scores(tokenized_query))
        
        rank,score= torch.sort(scores,descending=True).indices,torch.sort(scores,descending=True).values
        
        query_passage_score.append((query,rank[:k],score[:k]))
    
    return query_passage_score

In [ ]:
def get_relevant_doc_bm25L(clean_context_list,test_query_list,k):
    
    tokenized_context = [mecab_tokenize(context) for context in notebook.tqdm(clean_context_list)]
    
    bm25 = BM25L(tokenized_context)
    
    query_passage_score = []
        
    for query in notebook.tqdm(test_query_list):
        
        tokenized_query = mecab_tokenize(query)
        
        scores = torch.tensor(bm25.get_scores(tokenized_query))
        
        rank,score= torch.sort(scores,descending=True).indices,torch.sort(scores,descending=True).values
        
        query_passage_score.append((query,rank[:k],score[:k]))
    
    return query_passage_score

In [ ]:
def get_relevant_doc_bm25plus(clean_context_list,test_query_list,k):
    
    tokenized_context = [mecab_tokenize(context) for context in notebook.tqdm(clean_context_list)]
    
    bm25 = BM25Plus(tokenized_context)
    
    query_passage_score = []
        
    for query in notebook.tqdm(test_query_list):
        
        tokenized_query = mecab_tokenize(query)
        
        scores = torch.tensor(bm25.get_scores(tokenized_query))
        
        rank,score= torch.sort(scores,descending=True).indices,torch.sort(scores,descending=True).values
        
        query_passage_score.append((query,rank[:k],score[:k]))
    
    return query_passage_score

In [ ]:
query_passage_score_bm25 = get_relevant_doc_bm25(clean_context_list,test_query_list,k=10)

In [ ]:
query_passage_score_bm25L = get_relevant_doc_bm25L(clean_context_list,test_query_list,k=5)

In [ ]:
query_passage_score_bm25plus = get_relevant_doc_bm25plus(clean_context_list,test_query_list,k=5)

In [ ]:
query_passage_score_bm25[598]

('제2캐나다기갑여단이 상륙한 곳은?',
 tensor([53965, 38191, 28170, 38931,  5746, 10133, 46331, 16697, 45536, 52098,
         20085, 41922,  8714, 46860, 20205, 54101, 38220, 28314,  4636, 43249]),
 tensor([66.8892, 58.6919, 56.0010, 55.6272, 54.1833, 53.8530, 51.5529, 50.9585,
         50.5867, 49.2934, 46.2545, 45.7278, 45.0299, 43.0670, 42.4185, 40.9491,
         40.4398, 39.9208, 39.9062, 39.5890], dtype=torch.float64))

In [ ]:
query_passage_score_bm25L[598]

('제2캐나다기갑여단이 상륙한 곳은?',
 tensor([37116, 24529,  8699, 18919, 55238]),
 tensor([684.6677, 588.6879, 470.4238, 438.8907, 428.4706], dtype=torch.float64))

In [ ]:
query_passage_score_bm25plus[598]

('제2캐나다기갑여단이 상륙한 곳은?',
 tensor([10960,  2098, 21367, 26402, 51103, 45549, 17820, 17527, 20194, 31214,
          3646, 17652, 36330, 28979, 24950, 39127,  1413, 38588, 23120, 14529]),
 tensor([74.7134, 68.1146, 61.8993, 61.7824, 61.5399, 61.0321, 60.7731, 60.5152,
         60.3628, 60.0577, 59.1899, 58.7794, 58.6195, 58.3924, 57.4568, 56.7246,
         55.9776, 55.8603, 55.6965, 55.3557], dtype=torch.float64))

## concatenation sparse and dense passage vector

In [ ]:
k=50

query_passage_score = get_relevant_doc_concat(q_encoder,p_embs,sp_matrix,vectorizer,test_query_list,k)

In [ ]:
query_passage_score[598]

('제2캐나다기갑여단이 상륙한 곳은?',
 tensor([39532, 52694, 39519, 39518, 52695, 52874, 39592, 31628, 39535, 39533,
         52693, 52875, 39787, 39521, 39520, 39418, 39550, 39531, 39584, 19145,
         17525, 39790, 24709, 40358, 39789, 39514, 45772, 46098, 52877, 36835,
         52700, 49810, 39099, 53742, 39809, 52876, 40440, 18507, 37852, 39399,
         42259, 33445, 53739,  4473, 33444, 55349, 44417,  7797, 36150, 39587]),
 tensor([47.9594, 35.6455, 31.6161, 30.3620, 30.2213, 29.9415, 28.3232, 28.2927,
         27.8151, 27.2074, 27.0227, 25.9576, 25.8486, 25.2052, 22.6478, 21.7638,
         21.0442, 19.7188, 19.4975, 19.0694, 19.0117, 18.8128, 18.5482, 18.4788,
         18.4722, 18.2397, 17.9573, 16.5311, 16.2054, 16.1500, 15.9964, 15.7212,
         15.5857, 15.5220, 15.4210, 15.1123, 15.0782, 15.0356, 15.0154, 14.8798,
         14.8703, 14.6522, 14.4927, 14.2984, 13.9865, 13.9750, 13.9075, 13.8468,
         13.8140, 13.6262], dtype=torch.float64))

In [ ]:
def get_relevant_doc_dense_bm25(query_passage_score_bm25,query_passage_score_dense):
    
    query_passage_score=[]
    
    for a,b in notebook.tqdm(zip(query_passage_score_bm25,query_passage_score_dense)):
        
        query = a[0]
        
        bm25_passage = list(a[1][:15].numpy())
        dense_passage = list(b[1][:15].numpy())
        
        score = []
        score = torch.tensor(score)
        
        bm25_passage.extend(dense_passage)
        
        bm25_passage = torch.tensor(list(set(bm25_passage)))
        
        query_passage_score.append((query,bm25_passage,score))
        
    
    return query_passage_score

In [ ]:
query_passage_score = get_relevant_doc_dense_bm25(query_passage_score_bm25,query_passage_score_bm25plus)

In [ ]:
query_passage_score[300]

('루이가 사망하게 될 원인은 무엇인가?',
 tensor([ 4992, 38931, 33947, 54302, 35373, 42677, 39999, 24640, 42049,  9553,
         31313, 46294,    88, 11226,  2270, 41696, 10340, 40804, 22767, 52733]),
 tensor([]))

In [ ]:
#save passage embedding

with open('query_passage_score_k20_bm25.pkl', 'wb') as f:
    pickle.dump(query_passage_score_bm25, f)

In [ ]:
with open('query_passage_score_k20_bm25.pkl', 'rb') as f:
    query_passage_score = pickle.load(f)

# final inference by single passage method

In [ ]:
pad_on_right = mrc_tokenizer.padding_side == "right"

In [ ]:
k=50

In [ ]:
prediction_dict = final_inference(k,clean_context_list,query_passage_score)

# final ensemble inference by single passage method

In [ ]:
pad_on_right = mrc_tokenizer.padding_side == "right"

k=50

mrc_model1 = torch.load('mrc_model_koelectra_base_v3_discrim_full_aihub.pth').cuda()

mrc_model2 = torch.load('mrc_model_koelectra_base_v3_discrim_korquad_aihub_1epoch_retrain.pth').cuda()

mrc_model3 = torch.load('mrc_model_koelectra_base_v3_discrim_concat_retrain.pth').cuda()

mrc_model4 = torch.load('mrc_model_koelectra_base_v3_discrim_full_concat.pth').cuda()

mrc_model5 = torch.load('mrc_model_koelectra_base_v3_discrim_korquad_retrain.pth').cuda()

In [ ]:
prediction_dict = final_inference_ensemble_five(k,clean_context_list,query_passage_score)

# final prediction using multiple passage method

In [ ]:
k=20

query_passage_score = query_passage_score_bm25

In [ ]:
pad_on_right = mrc_tokenizer.padding_side == "right"

mrc_model = torch.load('mrc_model_koelectra_base_v3_discrim_aihub_korquad_noclean_retrain.pth').cuda()

prediction_dict = final_inference_multiple(clean_context_list,query_passage_score)

# final ensemble prediction using multiple passage method

In [ ]:
pad_on_right = mrc_tokenizer.padding_side == "right"

mrc_model1 = torch.load('mrc_model_koelectra_base_v3_discrim_full_aihub.pth').cuda()

mrc_model2 = torch.load('mrc_model_koelectra_base_v3_discrim_korquad_aihub_1epoch_retrain.pth').cuda()

mrc_model3 = torch.load('mrc_model_koelectra_base_v3_discrim_aihub_korquad_noclean_retrain.pth').cuda()

mrc_model4 = torch.load('mrc_model_koelectra_base_v3_discrim_korquad_retrain.pth').cuda()

In [ ]:
k=5

query_passage_score = query_passage_score_bm25plus

In [ ]:
prediction_dict = final_inference_ensemble_five_multiple(clean_context_list,query_passage_score)

# submission

In [ ]:
#remove score

final_dict = {}

for key,value in prediction_dict.items():
    
    final_dict[key] = value[0]

In [ ]:
with open('prediction.json', 'w', encoding='utf-8') as make_file:
    json.dump(final_dict, make_file, indent="\t")

wandb: Network error resolved after 0:01:01.939667, resuming normal operation.
